# Insert Python dataframe into Azure SQL table

## Import the libraries

In [1]:
import pyodbc
import pandas

## Open the connection

In [19]:
import pyodbc

DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = r'LAPTOP-D1GHCN4R\POWERBIREPORT'
DATABASE_NAME = 'AdventureWorksLT2014'
USERNAME = 'pbi'
PASSWORD = '123'

connection_string = f"""
DRIVER={{{DRIVER_NAME}}};
SERVER={SERVER_NAME};
DATABASE={DATABASE_NAME};
UID={USERNAME};
PWD={PASSWORD};
"""

conn = pyodbc.connect(connection_string)
print(conn)

## Create a cursor object

In [34]:
cursor = conn.cursor()

## Create a table

In [10]:
# Read data
cursor.execute("""SELECT top 5*
  FROM [AdventureWorksLT2014].[SalesLT].[Address]""")
rows = cursor.fetchall()

# Print data
for row in rows:
    print(row)

(9, '8713 Yosemite Ct.', None, 'Bothell', 'Washington', 'United States', '98011', '268AF621-76D7-4C78-9441-144FD139821A', datetime.datetime(2002, 7, 1, 0, 0))
(11, '1318 Lasalle Street', None, 'Bothell', 'Washington', 'United States', '98011', '981B3303-ACA2-49C7-9A96-FB670785B269', datetime.datetime(2003, 4, 1, 0, 0))
(25, '9178 Jumping St.', None, 'Dallas', 'Texas', 'United States', '75201', 'C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6', datetime.datetime(2002, 9, 1, 0, 0))
(28, '9228 Via Del Sol', None, 'Phoenix', 'Arizona', 'United States', '85004', '12AE5EE1-FC3E-468B-9B92-3B970B169774', datetime.datetime(2001, 9, 1, 0, 0))
(32, '26910 Indela Road', None, 'Montreal', 'Quebec', 'Canada', 'H1Y 2H5', '84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D', datetime.datetime(2002, 8, 1, 0, 0))


In [35]:

# SQL commands to drop the table if it exists and then create it
try:
    # Drop the previous table if it exists
    cursor.execute("DROP TABLE IF EXISTS [dbo].[movies]")

    # Create the table
    cursor.execute(
        """
        CREATE TABLE [dbo].[movies] (
            movie_id INT IDENTITY(1,1) PRIMARY KEY,
            movie_title VARCHAR(255) NOT NULL,
            released_year INT,
            runtime INT NOT NULL,
            genre VARCHAR(255),
            rating INT NOT NULL,
            director VARCHAR(255),
            star1 VARCHAR(255),
            star2 VARCHAR(255),
            number_of_votes INT,
            gross INT
        );
        """
    )

    # Commit the transaction
    conn.commit()
    print("Table created successfully.")

except Exception as e:
    print("Error:", e)

finally:
    # Close the cursor and connection
    # cursor.close()
    # conn.close()
    # print("Connection closed.")
    print("Connection.")


Table created successfully.
Connection.


## Open the file

In [23]:
fpath = "movies.CSV"
df = pandas.read_csv(fpath, delimiter=";", encoding="utf8")

In [24]:
df

,Title,Released_Year,Runtime,Genre,Rating,Director,Star1,Star2,Number_of_Votes,Gross
0,The Dark Knight,2008,152 min,"Action, Crime, Drama",90,Christopher Nolan,Christian Bale,Heath Ledger,2303232,"534,858,444"
1,The Lord of the Rings: The Return of the King,2003,201 min,"Action, Adventure, Drama",89,Peter Jackson,Elijah Wood,Viggo Mortensen,1642758,"377,845,905"
2,Inception,2010,148 min,"Action, Adventure, Sci-Fi",88,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,2067042,"292,576,195"
3,The Lord of the Rings: The Fellowship of the Ring,2001,178 min,"Action, Adventure, Drama",88,Peter Jackson,Elijah Wood,Ian McKellen,1661481,"315,544,750"
4,The Lord of the Rings: The Two Towers,2002,179 min,"Action, Adventure, Drama",87,Peter Jackson,Elijah Wood,Ian McKellen,1485555,"342,551,365"
...,...,...,...,...,...,...,...,...,...,...
296,28 Days Later...,2002,113 min,"Drama, Horror, Sci-Fi",76,Danny Boyle,Cillian Murphy,Naomie Harris,376853,"45,064,915"
297,The Royal Tenenbaums,2001,110 min,"Comedy, Drama",76,Wes Anderson,Gene Hackman,Gwyneth Paltrow,266842,"52,364,010"
298,Enemy at the Gates,2001,131 min,"Drama, History, War",76,Jean-Jacques Annaud,Jude Law,Ed Harris,243729,"51,401,758"
299,Minority Report,2002,145 min,"Action, Crime, Mystery",76,Steven Spielberg,Tom Cruise,Colin Farrell,508417,"132,072,926"


## Transform columns and change data types

### Runtime

In [25]:
def convert_runtime(x):
    """
    Convert the string value to int.
    - Remove "min".
    - Convert to int.
    """
    return int(x.split()[0])

In [26]:
df['Runtime'] = df['Runtime'].apply(convert_runtime)

In [27]:
df['Runtime']

0      152
1      201
2      148
3      178
4      179
      ... 
296    113
297    110
298    131
299    145
300    101
Name: Runtime, Length: 301, dtype: int64

### Genre

In [28]:
df['Genre'] = df['Genre'].apply(lambda x: x.split(', ')[0])

### Gross

In [29]:
def convert_gross(x):
    """
    Convert the string value to int.
    - Remove commas.
    - Convert to int.
    """
    return int(x.replace(',', ''))

In [30]:
df['Gross'] = df['Gross'].apply(convert_gross)

In [31]:
df

,Title,Released_Year,Runtime,Genre,Rating,Director,Star1,Star2,Number_of_Votes,Gross
0,The Dark Knight,2008,152,Action,90,Christopher Nolan,Christian Bale,Heath Ledger,2303232,534858444
1,The Lord of the Rings: The Return of the King,2003,201,Action,89,Peter Jackson,Elijah Wood,Viggo Mortensen,1642758,377845905
2,Inception,2010,148,Action,88,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,2067042,292576195
3,The Lord of the Rings: The Fellowship of the Ring,2001,178,Action,88,Peter Jackson,Elijah Wood,Ian McKellen,1661481,315544750
4,The Lord of the Rings: The Two Towers,2002,179,Action,87,Peter Jackson,Elijah Wood,Ian McKellen,1485555,342551365
...,...,...,...,...,...,...,...,...,...,...
296,28 Days Later...,2002,113,Drama,76,Danny Boyle,Cillian Murphy,Naomie Harris,376853,45064915
297,The Royal Tenenbaums,2001,110,Comedy,76,Wes Anderson,Gene Hackman,Gwyneth Paltrow,266842,52364010
298,Enemy at the Gates,2001,131,Drama,76,Jean-Jacques Annaud,Jude Law,Ed Harris,243729,51401758
299,Minority Report,2002,145,Action,76,Steven Spielberg,Tom Cruise,Colin Farrell,508417,132072926


In [32]:
df.dtypes

Title              object
Released_Year       int64
Runtime             int64
Genre              object
Rating              int64
Director           object
Star1              object
Star2              object
Number_of_Votes     int64
Gross               int64
dtype: object

## Insert data in the *movies* table

In [36]:
# Prepare the SQL statement
insert_query = """
INSERT INTO [dbo].[movies]
(movie_title, released_year, runtime, genre, rating, director, star1, star2, number_of_votes, gross)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Prepare data for bulk insert
data_to_insert = [
    (row['Title'], row['Released_Year'], row['Runtime'], row['Genre'], row['Rating'],
     row['Director'], row['Star1'], row['Star2'], row['Number_of_Votes'], row['Gross'])
    for _, row in df.iterrows()
]

# Enable fast_executemany for better performance
cursor.fast_executemany = True

# Execute bulk insert
cursor.executemany(insert_query, data_to_insert)

# Commit the transaction
conn.commit()

print(f"Successfully inserted {len(data_to_insert)} rows.")

Successfully inserted 301 rows.


## Commit the changes

In [37]:
conn.commit()

## Read data

In [38]:
# Read data
cursor.execute("SELECT "
    "movie_id, movie_title "
    "FROM [dbo].[movies]")
rows = cursor.fetchall()

# Print data
for row in rows:
  print(row)

(1, 'The Dark Knight')
(2, 'The Lord of the Rings: The Return of the King')
(3, 'Inception')
(4, 'The Lord of the Rings: The Fellowship of the Ring')
(5, 'The Lord of the Rings: The Two Towers')
(6, 'Interstellar')
(7, 'Joker')
(8, 'Whiplash')
(9, 'The Intouchables')
(10, 'The Prestige')
(11, 'The Departed')
(12, 'The Pianist')
(13, 'Gladiator')
(14, 'Spider-Man: Into the Spider-Verse')
(15, 'Avengers: Endgame')
(16, 'Avengers: Infinity War')
(17, 'Coco')
(18, 'Django Unchained')
(19, 'The Dark Knight Rises')
(20, '3 Idiots')
(21, 'Taare Zameen Par')
(22, 'WALL·E')
(23, 'The Lives of Others')
(24, 'Memento')
(25, '1917')
(26, 'Amelie')
(27, 'Snatch')
(28, 'Requiem for a Dream')
(29, 'Green Book')
(30, 'Drishyam')
(31, 'Queen')
(32, 'Warrior')
(33, 'Shutter Island')
(34, 'Up')
(35, 'The Wolf of Wall Street')
(36, 'Chak De! India')
(37, 'There Will Be Blood')
(38, "Pan's Labyrinth")
(39, 'Toy Story 3')
(40, 'V for Vendetta')
(41, 'Rang De Basanti')
(42, 'Black')
(43, 'Batman Begins')
(44

## Close the connection

In [39]:
cursor.close()
conn.close()